In [1]:
import pandas as pd 
import numpy as np 
import configparser
import os
import seaborn as sns
import plotly_express as px

import nltk 

config = configparser.ConfigParser()
config.read("env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
base_path = config['DEFAULT']['base_path']
code_dir = config['DEFAULT']['code_dir']

data_prefix = 'Maha'
out_path = f'{output_dir}/{data_prefix}'
OHCO = ['book_id','chap_id','sec_id','para_num', 'sent_num', 'token_num']


os.chdir(code_dir)
import preprocess
import bow_analysis
os.chdir(base_path)

In [2]:
CORPUS = pd.read_csv(rf"{output_dir}\F3\{data_prefix}-CORPUS.csv", sep="|").set_index(OHCO)
LIB = pd.read_csv(fr"{output_dir}\F3\{data_prefix}-LIB.csv", sep="|").set_index('book_id')
LIB2 = pd.read_csv(fr"{output_dir}\F3\{data_prefix}-LIB2.csv", sep="|").set_index(OHCO[:2])
VOCAB = pd.read_csv(rf"{output_dir}\F3\{data_prefix}-VOCAB.csv", sep="|").set_index("term_str")
VOCAB2 = pd.read_csv(rf"{output_dir}\CHAP_BOW\{data_prefix}-VOCAB2.csv", sep="|").set_index("term_str")
TFIDF = pd.read_csv(rf"{output_dir}\CHAP_BOW\{data_prefix}-TFIDF_REDUCED_CHAPS_L2.csv", sep="|").set_index(OHCO[:2])

In [3]:
TFIDF

shafts   pierced      army   deities  carwarriors  \
book_id chap_id                                                        
1       1        0.000000  0.017978  0.029045  0.000000     0.000000   
        2        0.000000  0.006462  0.037584  0.009168     0.000000   
        3        0.000000  0.000000  0.000000  0.013734     0.000000   
        4        0.000000  0.009420  0.000000  0.000000     0.000000   
        5        0.000000  0.020106  0.000000  0.032600     0.000000   
...                   ...       ...       ...       ...          ...   
15      2        0.000000  0.000000  0.000000  0.226051     0.013851   
        3        0.017530  0.000000  0.000000  0.000000     0.000000   
16      1        0.062153  0.014705  0.000000  0.041726     0.086291   
17      1        0.000000  0.000000  0.000000  0.201742     0.000000   
18      1        0.000000  0.000000  0.000000  0.296294     0.059203   

                   troops   sanjaya       cut  penances    partha  ...  \
book_id chap_id                                                    ...   
1       1        0.000000  0.674489  0.000000  0.000000  0.000000  ...   
        2        0.050112  0.049816  0.000000  0.000000  0.000000  ...   
        3        0.000000  0.000000  0.000000  0.000000  0.000000  ...   
        4        0.000000  0.000000  0.000000  0.022543  0.000000  ...   
        5        0.000000  0.000000  0.016690  0.099664  0.000000  ...   
...                   ...       ...       ...       ...       ...  ...   
15      2        0.030889  0.000000  0.000000  0.317740  0.000000  ...   
        3        0.000000  0.374090  0.000000  0.111660  0.000000  ...   
16      1        0.014255  0.000000  0.000000  0.035190  0.058821  ...   
17      1        0.000000  0.000000  0.000000  0.042535  0.017775  ...   
18      1        0.000000  0.000000  0.010552  0.043458  0.000000  ...   

                 propitiatory  apartment  anguish    thorn  cheered  \
book_id chap_id                                                       
1       1                 0.0   0.000000  0.01683  0.00000      0.0   
        2                 0.0   0.000000  0.00000  0.00000      0.0   
        3                 0.0   0.000000  0.00000  0.00000      0.0   
        4                 0.0   0.054643  0.00000  0.00000      0.0   
        5                 0.0   0.000000  0.00000  0.00833      0.0   
...                       ...        ...      ...      ...      ...   
15      2                 0.0   0.000000  0.00000  0.00000      0.0   
        3                 0.0   0.000000  0.00000  0.00000      0.0   
16      1                 0.0   0.000000  0.00000  0.00000      0.0   
17      1                 0.0   0.000000  0.00000  0.00000      0.0   
18      1                 0.0   0.000000  0.00000  0.00000      0.0   

                 believing     tops  pressing       hid    affair  
book_id chap_id                                                    
1       1              0.0  0.00000   0.00000  0.000000  0.000000  
        2              0.0  0.00000   0.00000  0.000000  0.000000  
        3              0.0  0.00000   0.00000  0.000000  0.000000  
        4              0.0  0.00000   0.00000  0.000000  0.027321  
        5              0.0  0.00833   0.00833  0.008066  0.000000  
...                    ...      ...       ...       ...       ...  
15      2              0.0  0.00000   0.00000  0.000000  0.000000  
        3              0.0  0.00000   0.00000  0.000000  0.000000  
16      1              0.0  0.00000   0.00000  0.000000  0.000000  
17      1              0.0  0.00000   0.00000  0.000000  0.000000  
18      1              0.0  0.00000   0.00000  0.000000  0.000000  

[99 rows x 3080 columns]

In [4]:
from sklearn.decomposition import PCA
n_comps = 5


pc_cols = [f"PC{i}" for i in range(n_comps)]
pca_engine = PCA(n_components=n_comps)
DCM = pd.DataFrame(pca_engine.fit_transform(TFIDF), index=TFIDF.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM

PC0       PC1       PC2       PC3       PC4
book_id chap_id                                                  
1       1        0.015656 -0.035860 -0.300592  0.175487  0.185307
        2       -0.105716  0.035544 -0.023812  0.004266  0.071444
        3       -0.173461  0.063764  0.079875 -0.066908 -0.076211
        4       -0.181235  0.189262  0.270950  0.239522  0.025505
        5       -0.150630 -0.041170  0.162680 -0.077649 -0.113647
...                   ...       ...       ...       ...       ...
15      2       -0.179489 -0.257950 -0.000092  0.063901  0.236936
        3       -0.052523 -0.109749 -0.251448  0.210425  0.273081
16      1        0.008625  0.012658  0.008085 -0.158933  0.246950
17      1       -0.147082 -0.083067  0.063168 -0.093624  0.203213
18      1       -0.193000 -0.313920  0.145796 -0.093820  0.143602

[99 rows x 5 columns]

In [5]:
DCM = DCM.join(LIB2[['book_name','chap_name']], on=OHCO[:2])
DCM = DCM.join(LIB, on=OHCO[:1])
DCM

PC0       PC1       PC2       PC3       PC4  \
book_id chap_id                                                     
1       1        0.015656 -0.035860 -0.300592  0.175487  0.185307   
        2       -0.105716  0.035544 -0.023812  0.004266  0.071444   
        3       -0.173461  0.063764  0.079875 -0.066908 -0.076211   
        4       -0.181235  0.189262  0.270950  0.239522  0.025505   
        5       -0.150630 -0.041170  0.162680 -0.077649 -0.113647   
...                   ...       ...       ...       ...       ...   
15      2       -0.179489 -0.257950 -0.000092  0.063901  0.236936   
        3       -0.052523 -0.109749 -0.251448  0.210425  0.273081   
16      1        0.008625  0.012658  0.008085 -0.158933  0.246950   
17      1       -0.147082 -0.083067  0.063168 -0.093624  0.203213   
18      1       -0.193000 -0.313920  0.145796 -0.093820  0.143602   

                             book_name              chap_name  \
book_id chap_id                                                 
1       1                    Adi Parva     Anukramanika Parva   
        2                    Adi Parva         Sangraha Parva   
        3                    Adi Parva          Paushya Parva   
        4                    Adi Parva          Pauloma Parva   
        5                    Adi Parva           Astika Parva   
...                                ...                    ...   
15      2           Asramavasika Parva     Putradarsana Parva   
        3           Asramavasika Parva     Naradagamana Parva   
16      1                Mausala Parva          Mausala Parva   
17      1        Mahaprasthanika Parva  Mahaprasthanika Parva   
18      1         Svargarohanika Parva   Svargarohanika Parva   

                source_file_path              raw_title  book_len  n_chaps  \
book_id chap_id                                                              
1       1             maha01.txt              Adi Parva    228684       19   
        2             maha01.txt              Adi Parva    228684       19   
        3             maha01.txt              Adi Parva    228684       19   
        4             maha01.txt              Adi Parva    228684       19   
        5             maha01.txt              Adi Parva    228684       19   
...                          ...                    ...       ...      ...   
15      2             maha15.txt     Asramavasika Parva     30376        3   
        3             maha15.txt     Asramavasika Parva     30376        3   
16      1             maha16.txt          Mausala Parva      8072        1   
17      1             maha17.txt  Mahaprasthanika Parva      3062        1   
18      1             maha18.txt   Svargarohanika Parva      8628        1   

                 n_secs  n_sents  n_tokens  n_chars       parva_name  \
book_id chap_id                                                        
1       1           235    12422    228684  1055885              Adi   
        2           235    12422    228684  1055885              Adi   
        3           235    12422    228684  1055885              Adi   
        4           235    12422    228684  1055885              Adi   
        5           235    12422    228684  1055885              Adi   
...                 ...      ...       ...      ...              ...   
15      2            38     1742     30376   141565    Ashramavasika   
        3            38     1742     30376   141565    Ashramavasika   
16      1             7      432      8072    37596          Mausala   
17      1             2      206      3062    14086  Mahaprasthanika   
18      1             5      495      8628    40381     Svargarohana   

                 timeline                                          key_focus  \
book_id chap_id                                                                
1       1         Pre-War      Origins, family lineages, childhood of heroes   
        2         Pre-War      Origins, family lineages, childhood of heroes   
      

In [6]:
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.index = TFIDF.columns
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index.name = 'term_str'
LOADINGS2 = LOADINGS.join(VOCAB2, on="term_str")
LOADINGS2

,PC0,PC1,PC2,PC3,PC4,Unnamed: 0,n,n_chars,p,i,...,n_pos,cat_pos,df,idf,dp,di,dh,tfidf_mean,tfidf_sum,dfidf
term_str,,,,,,,,,,,,,,,,,,,,,
shafts,0.081491,-0.008007,0.016090,-0.004886,-0.008915,23900,2704,6,0.001122,9.799383,...,11,"{'NNS', 'VBN', 'JJ', 'VBP', 'VBD', 'VBZ', 'NN'...",57.0,0.796467,0.575758,0.796467,0.458572,0.000569,0.056298,45.398597
pierced,0.050604,-0.006929,0.010585,-0.001266,-0.005272,19808,1637,7,0.000679,10.523424,...,5,"{'VBN', 'JJ', 'VBD', 'NN', 'NNP'}",55.0,0.847997,0.555556,0.847997,0.471109,0.000360,0.035682,46.639830
army,0.038255,-0.007283,-0.005226,0.000643,-0.001683,3462,1332,4,0.000553,10.820884,...,8,"{'NNS', 'JJ', 'VBP', 'IN', 'VBZ', 'VBD', 'NN',...",56.0,0.822002,0.565657,0.822002,0.464971,0.000389,0.038510,46.032095
deities,-0.019493,-0.043460,0.020318,-0.010743,0.014481,8090,1315,7,0.000546,10.839415,...,5,"{'NNS', 'JJ', 'NNPS', 'VBZ', 'NNP'}",43.0,1.203092,0.434343,1.203092,0.522555,0.000281,0.027864,51.732950
carwarriors,0.054079,-0.008154,0.001406,0.003460,0.003115,5610,1286,11,0.000534,10.871587,...,2,"{'NNP', 'NNS'}",46.0,1.105795,0.464646,1.105795,0.513804,0.000441,0.043627,50.866555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
believing,-0.000404,-0.000234,-0.000491,0.000435,-0.001068,4439,22,9,0.000009,16.740831,...,2,"{'VBG', 'NN'}",18.0,2.459432,0.181818,2.459432,0.447169,0.000023,0.002284,44.269769
tops,-0.000454,-0.000040,-0.001456,0.000336,-0.000279,26987,21,4,0.000009,16.807945,...,1,{'NNS'},18.0,2.459432,0.181818,2.459432,0.447169,0.000036,0.003612,44.269769
pressing,0.000153,0.003229,0.001077,0.003733,0.001743,20508,20,8,0.000008,16.878334,...,1,{'VBG'},18.0,2.459432,0.181818,2.459432,0.447169,0.000055,0.005441,44.269769


In [10]:
def vis_pcs(M, a, b,color ="book_name" ,hover_name='chap_name'):
    M["book_name"] = pd.Categorical(M["book_name"], ordered=True)
    return px.scatter(M, f"PC{a}", f"PC{b}", 
                      color = color ,
                    hover_name=hover_name, 
                    # symbol=symbol, size=size,
                    marginal_x='box', height=800, width = 1008)

def vis_loadings(LOADINGS, a=0, b=1, hover_name='term_str'):
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', hover_name='term_str',
                       color='max_pos_group', size="dfidf",
                      marginal_x='box', height=800)

In [11]:
vis_pcs(DCM, 0, 1, "timeline")

In [12]:
vis_loadings(LOADINGS2, 0, 1)

In [13]:
vis_pcs(DCM, 1, 2, "timeline")

In [14]:
vis_loadings(LOADINGS2, 1, 2)

In [16]:
vis_pcs(DCM, 2, 3, "timeline")

In [17]:
vis_loadings(LOADINGS2, 2,3)

In [18]:
top_terms_sk= {}

data = []
for i in range(n_comps):
    for j in [0, 1]:
        data.append((f"PC{i}", j, ' '.join(LOADINGS.sort_values(f'PC{i}', ascending=bool(j)).head(10).index.to_list())))

comp_strs = pd.DataFrame(data)
comp_strs.columns =  ['pc', 'pole', 'top_terms']
comp_strs = comp_strs.set_index(['pc', 'pole'])


COMPS = comp_strs
COMPS['top_doc'] = pd.concat([DCM[pc_cols].idxmax(), DCM[pc_cols].idxmin()], axis=0, keys=[0,1])\
    .to_frame('top_doc')\
    .reset_index()\
    .rename(columns={'level_0':'pole', 'level_1':'pc'})\
    .set_index(['pc','pole'])\
    .sort_index()

In [19]:
COMPS

top_terms  top_doc
pc  pole                                                            
PC0 0     shafts carwarriors pierced sanjaya army cut dr...   (7, 6)
    1     deities vidura penances vyasa brahman becomes ...  (13, 1)
PC1 0     rakshasa jarasandha monarchs virata sisupala r...   (3, 2)
    1     deities righteousness becomes penances brahman...  (12, 3)
PC2 0     rakshasa deities shafts bhrigu rama vasishtha ...   (1, 9)
    1     sanjaya vidura gandhari dussasana hope dhritar...   (5, 2)
PC3 0     rakshasa sanjaya vidura gandhari sleeping hope...   (3, 2)
    1     jarasandha sisupala monarchs vrishni partha an...  (1, 18)
PC4 0     jarasandha sanjaya rakshasa magadha gandhari d...   (2, 3)
    1     virata dice spake play vidura dussasana matsya...   (3, 6)

In [20]:
TFIDF.loc[(7, 6)].sort_values(ascending=False).head(20)

term_str
shafts         0.389722
satyaki        0.316299
pierced        0.219966
sindhus        0.217252
carwarriors    0.165149
cut            0.146215
sanjaya        0.145163
sini           0.144992
yuyudhana      0.133321
bhurisravas    0.123660
keen           0.118353
grandson       0.116777
shot           0.115106
satwata        0.108736
host           0.107483
rushed         0.104546
carwarrior     0.101800
showers        0.099959
partha         0.097485
kritavarman    0.096659
Name: (7, 6), dtype: float64

In [21]:
comp_strs.unstack()

top_terms  \
pole                                                  0   
pc                                                        
PC0   shafts carwarriors pierced sanjaya army cut dr...   
PC1   rakshasa jarasandha monarchs virata sisupala r...   
PC2   rakshasa deities shafts bhrigu rama vasishtha ...   
PC3   rakshasa sanjaya vidura gandhari sleeping hope...   
PC4   jarasandha sanjaya rakshasa magadha gandhari d...   

                                                        top_doc           
pole                                                  1       0        1  
pc                                                                        
PC0   deities vidura penances vyasa brahman becomes ...  (7, 6)  (13, 1)  
PC1   deities righteousness becomes penances brahman...  (3, 2)  (12, 3)  
PC2   sanjaya vidura gandhari dussasana hope dhritar...  (1, 9)   (5, 2)  
PC3   jarasandha sisupala monarchs vrishni partha an...  (3, 2)  (1, 18)  
PC4   virata dice spake play vidura dussasana matsya...  (2, 3)   (3, 6)